# Experimento Doc2Vec - TFIDF - Embeddings

O objetivo aqui é usar doc2vec para classificação multi-classe no mini-dataset de atos/classes gerado após teste no ezTag. O resultado será comparado com representações textuais TFIDF e embeddings diversas; todos os casos serão testados com SVM e Logistic Regression.

Fontes principais:
- https://towardsdatascience.com/implementing-multi-class-text-classification-with-doc2vec-df7c3812824d
- https://github.com/UnB-KnEDLe/experiments/blob/master/members/matheus/BioC_XML_to_conll.ipynb
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

Os testes foram feitos no Colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import de packages

In [1]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
from tqdm import tqdm
import multiprocessing
import nltk
import numpy as np

## Mini-Dataset de atos
- Instanciação em memória
- Retirada de classes com apenas 1 instância

In [2]:
import pandas as pd

df = pd.read_csv('/content/atos_segs.csv')
df

,tipo_ato,ato
0,nomeacao,NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o ...
1,exoneracao,"EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY d..."
2,nomeacao,NOMEAR SANDRA TURCATO JORGE TOLENTINO para exe...
3,nomeacao,"NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurad..."
4,ato_tornado_sem_efeito,TORNAR SEM EFEITO no Decreto de 02 de julho de...
...,...,...
268,substituicao,DESIGNAR\nrespectivamente TEREZA CRISTINA DE A...
269,substituicao,"Designar FLÁVIO DE ARAÚJO ALMEIDA, matrícula n..."
270,substituicao,"DESIGNAR OTAMÁ DANTAS BARRETO, matrícula\n159...."
271,retificacao,"RETIFICAR o Sexto Termo\nAditivo ao Contrato, ..."


In [3]:
df.tipo_ato.value_counts()

aposentadoria             69
nomeacao                  53
exoneracao                49
retificacao               38
substituicao              37
ato_tornado_sem_efeito    16
cessao                     9
abono_de_permanência       1
reversao                   1
Name: tipo_ato, dtype: int64

In [4]:
df = df[df.tipo_ato != 'abono_de_permanência']
df = df[df.tipo_ato != 'reversao']
df.tipo_ato.value_counts()

aposentadoria             69
nomeacao                  53
exoneracao                49
retificacao               38
substituicao              37
ato_tornado_sem_efeito    16
cessao                     9
Name: tipo_ato, dtype: int64

### Criação de listas de atos e tipos de ato, em ordem

In [5]:
atos = df.ato.to_list()
tipos = df.tipo_ato.to_list()

In [6]:
len(atos), len(tipos)

(271, 271)

### Retira \n das instâncias de atos

In [7]:
import re

at_contiguos = []
for ato in atos:
    at_contiguos.append(re.sub('\n', ' ', ato))
#at_contiguos

In [8]:
len(at_contiguos)

271

### Cria lista de listas de referência

infos é uma lista que contém duas outras listas:
- tipos
- at_contiguos

Ela servirá de referência para mapear instâncias

In [9]:
infos = []
infos.append(tipos)
infos.append(at_contiguos)
infos

infos[0][1]

'exoneracao'

In [10]:
i = 0
infos[0][i], infos[1][i]

('nomeacao',
 'NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o Cargo em Comissão, Símbolo CC-04, código SIGRH B0001615, de Assessor Técnico, da Chefia de Gabinete, do Gabinete do Governador.')

### Função para tokenização

In [11]:
tqdm.pandas(desc="progress-bar")
# Function for tokenizing
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

### Mapeia labels

In [12]:
possible_labels = df.tipo_ato.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [13]:
df['label'] = df.tipo_ato.replace(label_dict)
df

,tipo_ato,ato,label
0,nomeacao,NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o ...,0
1,exoneracao,"EXONERAR, a pedido, ANNA LUIZA BARCIA HALLEY d...",1
2,nomeacao,NOMEAR SANDRA TURCATO JORGE TOLENTINO para exe...,0
3,nomeacao,"NOMEAR RAIMUNDO DA COSTA SANTOS NETO, Procurad...",0
4,ato_tornado_sem_efeito,TORNAR SEM EFEITO no Decreto de 02 de julho de...,2
...,...,...,...
268,substituicao,DESIGNAR\nrespectivamente TEREZA CRISTINA DE A...,4
269,substituicao,"Designar FLÁVIO DE ARAÚJO ALMEIDA, matrícula n...",4
270,substituicao,"DESIGNAR OTAMÁ DANTAS BARRETO, matrícula\n159....",4
271,retificacao,"RETIFICAR o Sexto Termo\nAditivo ao Contrato, ...",3


### Divisão treino e teste para fins de indexação do modelo doc2vec

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.ato.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=14, 
                                                  stratify=df.label.values)

In [ ]:
#X_train, X_test, y_train, y_test

In [15]:
infos[0][0], infos[1][0], len(infos[0])

('nomeacao',
 'NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o Cargo em Comissão, Símbolo CC-04, código SIGRH B0001615, de Assessor Técnico, da Chefia de Gabinete, do Gabinete do Governador.',
 271)

In [16]:
i = 0
infos[0][i], infos[1][i]

('nomeacao',
 'NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o Cargo em Comissão, Símbolo CC-04, código SIGRH B0001615, de Assessor Técnico, da Chefia de Gabinete, do Gabinete do Governador.')

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
# Initializing the variables
train_documents = []
test_documents = []

j = 0
for i in X_train:
    tag = y_train[j].item()
    train_documents.append(TaggedDocument(words=tokenize_text(i), tags=[tag]))
    j+=1

j = 0
for i in X_test:
    tag = y_test[j].item()
    test_documents.append(TaggedDocument(words=tokenize_text(i), tags=[tag]))
    j+=1

In [19]:
train_documents[7]

TaggedDocument(words=['exonerar', 'por', 'estar', 'sendo', 'nomeada', 'para', 'outro', 'cargo', 'karla', 'neres', 'de', 'laet', 'santana', 'do', 'cargo', 'em', 'comissão', 'símbolo', 'cc-06', 'código', 'sigrh', '07200232', 'de', 'assessor', 'do', 'gabinete', 'da', 'administração', 'regional', 'do', 'plano', 'piloto', 'do', 'distrito', 'federal'], tags=[1])

## doc2vec
Instanciação

In [20]:
cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=1, vector_size=100, negative=5, hs=0, min_count=2, sample = 0, workers=cores, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in tqdm(train_documents)])
train_documents  = utils.shuffle(train_documents)
model_dbow.train(train_documents,total_examples=len(train_documents), epochs=30)

def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags, model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors

#model_dbow.save('./actModel.d2v')

100%|██████████| 203/203 [00:00<00:00, 29627.80it/s]


In [21]:
len(train_documents)

203

Divisão treino e teste para uso no modelo em si

In [22]:
y_train_model, X_train_model = vector_for_learning(model_dbow, train_documents)
y_test_model, X_test_model = vector_for_learning(model_dbow, test_documents)

As duas células abaixo são usadas para prevenir o seguinte: `DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().`
O método ravel() converte o shape do array para (n,). Ainda assim o warning persiste!

In [23]:
y_resampled = pd.DataFrame(y_test_model)

In [24]:
vai = y_resampled.values
vai = vai.ravel()
vai.shape

(68,)

Utilização do modelo com LogisticRegression e SVM

In [25]:
logreg = LogisticRegression(max_iter=300000)
logreg.fit(X_train_model, y_train_model)
y_pred_d2v_reg = logreg.predict(X_test_model)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
from sklearn.svm import LinearSVC

svm_model_d2v = LinearSVC(max_iter=10000)
svm_model_d2v.fit(X_train_model, y_train_model)
svm_prediction_d2v = svm_model_d2v.predict(X_test_model)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## TFIDF

O primeiro passo é instanciar um objeto Tfidfvectorizer e gerar os vetores para o nosso corpus. A lista infos, que geramos algumas células atrás, é uma lista de 2 listas, que indexam em ordem label/texto de ato. Para gerar os vetores, vamos usar a lista dos textos de atos, infos[1]. O método fit_transform gera os vetores tfidf dos textos e os retorna em uma matriz esparsa. Embora os resultados tenham se apresentado idênticos em um experimento anterior 'primeiro usando fit_transform em todo o conjunto de dados e depois divindo em treino/teste', vamos adotar o padrão 'primeiro dividir em treino/teste, fit_transform nos dados de treino e apenas fit nos dados de teste', apenas por desencargo de consciência.
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [27]:
X_train, X_test, y_train, y_test = train_test_split(infos[1], df['label'], test_size=0.25, random_state=14, stratify=df['label'])

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# fit_transform no treino
X_csr_train = vectorizer.fit_transform(X_train)
print(X_csr_train.shape)

# apenas transform no teste
X_csr_test = vectorizer.transform(X_test)
print(X_csr_test.shape)

(203, 1651)
(68, 1651)


Com treino e teste em mãos, basta treinar os modelos.

TFIDF + SVM

In [31]:
svm_model = LinearSVC(max_iter=1000)
svm_model.fit(X_csr_train, y_train)
svm_prediction_tfidf = svm_model.predict(X_csr_test)

TFIDF + regressão

In [32]:
tfidf_logreg = LogisticRegression()
tfidf_logreg.fit(X_csr_train, y_train)
y_pred_tfidf_reg = tfidf_logreg.predict(X_csr_test)

Para os labels, vamos usar a coluna label do dataframe, gerada ao mapear os valores de infos[0], a lista de labels em ordem, para valores inteiros em um dict.

In [33]:
labels = df['label']
labels.shape

(271,)

In [39]:
from sklearn.metrics import precision_recall_fscore_support

## Embeddings

Variáveis de paths

In [34]:
path_w2v_cbow300 = '/content/drive/My Drive/pesquisa/embeddings_NILC_W2V/cbow_s300.txt'
path_w2v_skip300 = '/content/drive/My Drive/pesquisa/embeddings_NILC_W2V/skip_s300.txt'
path_glove300 = '/content/drive/My Drive/pesquisa/embeddings_NILC_GLOVE/glove_s300.txt'
path_ft_cbow300 = '/content/drive/My Drive/pesquisa/embeddings_NILC_FT/cbow_s300.txt'
path_ft_skip300 = '/content/drive/My Drive/pesquisa/embeddings_NILC_FT/skip_s300.txt'

Para cada tipo de embedding pré-treinado que temos, vamos instanciar um dicionário e fazer append dos 10000 primeiros (ou mais frequentes) termos como chaves e listas contendo as representações vetoriais desse termo. No caso, todas as embeddings que estamos usando tem 300 dimensões, então são 300 valores float em cada lista representando cada termo.

In [71]:
dictionary = open(path_ft_skip300, 'r', encoding='utf-8',
                  newline='\n', errors='ignore')
embeds = {}
for line in dictionary:
    tokens = line.rstrip().split(' ')
    embeds[tokens[0]] = [float(x) for x in tokens[1:]]
    
    if len(embeds) == 10000:
        break

print(len(embeds))

10000


Essa é uma exploração sobre a média de palavras em cada ato de nosso conjunto de dados. Com base nessa média vamos determinar o tamanho do slice na lista referente a cada ato já tokenizado.

In [37]:
print(infos[1][0])
print(f'Qtd de palavras doc 4: {len(infos[1][4])}')
soma = 0
for i in range(len(infos[1])):
  soma += len(infos[1][i])
media = soma/len(infos[1])
print(f'Média de palavras por ato: {media}')

NOMEAR LETÍCIA MATOS MAGALHÃES para exercer o Cargo em Comissão, Símbolo CC-04, código SIGRH B0001615, de Assessor Técnico, da Chefia de Gabinete, do Gabinete do Governador.
Qtd de palavras doc 4: 382
Média de palavras por ato: 420.8228782287823


Cada ato tem, em média, 420 palavras. Vamos, então, usar um slice dos 100 primeiros termos de cada ato para montar a representação embedding do nosso conjunto de dados. O pedaço de código abaixo é o seguinte:
1. Criamos um dataframe;
2. Tokenizamos e salvamos os 100 primeiros termos de cada ato em uma variável words; words será da forma ['nomear', 'fulano', 'de', 'tal', 'para', ...];
3. Para cada termo em words verificamos sua representação vetorial correspondente no dict que criamos mais cedo. Se o termo não existe no dict, apenas vamos para o próximo;
4. Supondo que todos os 100 termos existam no dict, nossa linha terá um shape (1, 30000), já que cada um dos 100 termos tem 300 dimensões em sua representação vetorial. Nos casos onde temos menos de 100 termos adicionados - quando o termo em questão não existe no dict, por exemplo - teríamos um shape diferente, e isso nos traria problemas no treinamento. Para resolver essa questão, salvamos em uma variável array_length o tamanho esperado de cada linha do dataframe e adicionamos zeros às linhas que não adicionarem nativamente os 100 termos, de modo que, adicionando ou não 100 termos, o shape será sempre (1, 30000);
5. Fazemos append de cada linha processada no dataframe criado.

In [72]:
# 100 tokens * 300 dimensões
array_length = 100 * 300
embedding_features = pd.DataFrame()
for document in infos[1]:
    # Saving the first 100 words of the document as a sequence
    words = tokenize_text(document)[0:100]
    
    # Retrieving the vector representation of each word and 
    # appending it to the feature vector 
    feature_vector = []
    for word in words:
        try:
            feature_vector = np.append(feature_vector, 
                                       np.array(embeds[word]))
        except KeyError:
            # In the event that a word is not included in our 
            # dictionary skip that word
            pass
    # If the text has less then 100 words, fill remaining vector with
    # zeros
    zeroes_to_add = array_length - len(feature_vector)
    if zeroes_to_add > 0:
      feature_vector = np.append(feature_vector, 
                                np.zeros(zeroes_to_add)
                                ).reshape((1,-1))
    
    feature_vector = feature_vector.reshape((1,-1))
    
    # Append the document feature vector to the feature table
    embedding_features = embedding_features.append( 
                                     pd.DataFrame(feature_vector))
print(embedding_features.shape)

(271, 30000)


Aqui fazemos o split do dataframe de embeddings em treino e teste. Mantemos as mesmas configurações dos testes anteriores, para fins de comparação.

In [73]:
emb_train, emb_test = train_test_split(embedding_features, test_size=0.25, random_state=14, stratify=labels)

print(emb_train.shape, emb_test.shape)
print(y_test.shape)

(203, 30000) (68, 30000)
(68,)


Com os conjuntos de treino e teste e respectivos rótulos já em mãos, basta rodar em seu modelo favorito.

Word2vec CBOW 300

In [41]:
svm_w2v_cbow300 = LinearSVC(max_iter=40000)
svm_w2v_cbow300.fit(emb_train, y_train)
svm_pred_w2v_cbow300 = svm_w2v_cbow300.predict(emb_test)

In [42]:
w2v_cbow300_logreg = LogisticRegression()
w2v_cbow300_logreg.fit(emb_train, y_train)
y_pred_w2v_cbow300_reg = w2v_cbow300_logreg.predict(emb_test)

Word2vec SkipGram 300

In [51]:
svm_w2v_skip300 = LinearSVC(max_iter=40000)
svm_w2v_skip300.fit(emb_train, y_train)
svm_pred_w2v_skip300 = svm_w2v_skip300.predict(emb_test)

In [52]:
w2v_skip300_logreg = LogisticRegression()
w2v_skip300_logreg.fit(emb_train, y_train)
y_pred_w2v_skip300_reg = w2v_skip300_logreg.predict(emb_test)

Glove 300

In [58]:
svm_glove300 = LinearSVC(max_iter=40000)
svm_glove300.fit(emb_train, y_train)
svm_pred_glove300 = svm_glove300.predict(emb_test)

In [60]:
glove300_logreg = LogisticRegression(max_iter=10000)
glove300_logreg.fit(emb_train, y_train)
y_pred_glove300_reg = glove300_logreg.predict(emb_test)

Fast-text CBOW 300

In [66]:
svm_ft_cbow300 = LinearSVC(max_iter=40000)
svm_ft_cbow300.fit(emb_train, y_train)
svm_pred_ft_cbow300 = svm_ft_cbow300.predict(emb_test)

In [68]:
ft_cbow300_logreg = LogisticRegression(max_iter=10000)
ft_cbow300_logreg.fit(emb_train, y_train)
y_pred_ft_cbow300_reg = ft_cbow300_logreg.predict(emb_test)

Fast-text SkipGram 300

In [74]:
svm_ft_skip300 = LinearSVC(max_iter=40000)
svm_ft_skip300.fit(emb_train, y_train)
svm_pred_ft_skip300 = svm_ft_skip300.predict(emb_test)

In [80]:
ft_skip300_logreg = LogisticRegression(max_iter=10000)
ft_skip300_logreg.fit(emb_train, y_train)
y_pred_ft_skip300_reg = ft_skip300_logreg.predict(emb_test)

## Resultados

In [44]:
from sklearn.metrics import precision_recall_fscore_support

In [81]:
results = pd.DataFrame(columns = ['Precision', 'Recall', 'F1 score', 'support']
          )
results.loc['d2v + regressão'] = precision_recall_fscore_support(
          vai, 
          y_pred_d2v_reg, 
          average = 'weighted'
          )
results.loc['d2v + SVM'] = precision_recall_fscore_support(
          vai, 
          svm_prediction_d2v, 
          average = 'weighted'
          )
results.loc['TFIDF + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_prediction_tfidf, 
          average = 'weighted'
          )
results.loc['TFIDF + regressão'] = precision_recall_fscore_support(
          y_test, 
          y_pred_tfidf_reg, 
          average = 'weighted'
          )
results.loc['W2V CBOW 300 + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_w2v_cbow300, 
          average = 'weighted'
          )
results.loc['W2V CBOW 300 + regressão'] = precision_recall_fscore_support(
          y_test, 
          y_pred_w2v_cbow300_reg, 
          average = 'weighted'
          )
results.loc['W2V Skip 300 + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_w2v_skip300, 
          average = 'weighted'
          )
results.loc['W2V Skip 300 + regressão'] = precision_recall_fscore_support(
          y_test, 
          y_pred_w2v_skip300_reg, 
          average = 'weighted'
          )
results.loc['Glove 300 + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_glove300, 
          average = 'weighted'
          )
results.loc['Glove 300 + regressão'] = precision_recall_fscore_support(
          y_test, 
          y_pred_glove300_reg, 
          average = 'weighted'
          )
results.loc['Fast-text CBOW 300 + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_ft_cbow300, 
          average = 'weighted'
          )
results.loc['Fast-text CBOW 300 + regressão'] = precision_recall_fscore_support(
          y_test, 
          y_pred_ft_cbow300_reg, 
          average = 'weighted'
          )
results.loc['Fast-text Skip 300 + SVM'] = precision_recall_fscore_support(
          y_test, 
          svm_pred_ft_skip300, 
          average = 'weighted'
          )
results.loc['Fast-text Skip 300 + regressão'] = precision_recall_fscore_support(
          y_test, 
          y_pred_ft_skip300_reg, 
          average = 'weighted'
          )

In [82]:
results

,Precision,Recall,F1 score,support
d2v + regressão,0.826824,0.838235,0.827509,NaN
d2v + SVM,0.877245,0.882353,0.878037,NaN
TFIDF + SVM,0.986631,0.985294,0.984594,NaN
TFIDF + regressão,0.972976,0.970588,0.966309,NaN
W2V CBOW 300 + SVM,0.986425,0.985294,0.985156,NaN
W2V CBOW 300 + regressão,0.959195,0.955882,0.956218,NaN
W2V Skip 300 + SVM,1.000000,1.000000,1.000000,NaN
W2V Skip 300 + regressão,0.971639,0.970588,0.970428,NaN
Glove 300 + SVM,0.986425,0.985294,0.985201,NaN
Glove 300 + regressão,0.943439,0.941176,0.941765,NaN


## Discussão

O uso de embeddings pré-treinadas aqui se mostrou muito efetivo. A princípio, eu havia carregado um vocabulário de 100000 termos. Quando rodei o experimento com SVM, o modelo acertou tudo. Diminuí o vocabulário para 10000 termos e os modelos passaram a acertar algo acima de 95% dos casos. Eu havia feito um experimento parecido com os dados do trabalho do Pedro Luz, sobre textos de secretarias, e havia obtido um valor menor de F1-score usando embeddings em relação ao baseline TFIDF. A princípio, então, pensei que o que estava acontecendo aqui era um erro XD, mas depois de refletir um pouco sobre a natureza da minha base de dados, imagino o seguinte cenário:
1. Os atos que constam no DODF tem uma estrutura muito mais rígida em relação aos textos de secretarias;
2. Outro fator que conta bastante é o vocabulário diminuto da minha base em relação à base do Pedro: lá eu usei as mesmas embeddings, com 100000 termos carregados no dict, e ainda assim tive perfomance menor que o baseline; aqui, usei 10% do número de termos e obtive performance semelhante ou melhor.